Mongolian text classification series #01

In this notebook I'm gonna try to classify cyrillic mongolian texts with 1D convnet. In this case, architecture from [official Keras documention](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) will be replicated and also functional API used.

Eduge dataset provided by Bolorsoft LLC

Author : Sharavsambuu Gunchinish (sharavsambuu@gmail.com)

Github: https://github.com/sharavsambuu/mongolian-text-classification 



In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-alpha0
!pip install gensim

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.0.0-alpha0


[More info about creation of eduge dataset pickles](https://github.com/sharavsambuu/mongolian-text-classification/blob/master/preprocess_dataset/preprocess_eduge.ipynb) preprocessing eats a lot of CPU cycle so it's good idea to cook it before using colab.

In [2]:
import os
from os.path import exists, join, basename, splitext
import sys

def download_from_google_drive(file_id, file_name):
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  
# download eduge pickles
file_path = 'eduge_pickles'
if not exists(file_path):
  download_from_google_drive('1vjJ9YgIe8o0ErhbN0lH1XqPv3KFP8acv', '%s.rar' % file_path)
  rar_file = file_path+".rar"
  !unrar x $rar_file

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   7461      0 --:--:-- --:--:-- --:--:--  7461
100  106M    0  106M    0     0   112M      0 --:--:-- --:--:-- --:--:--  150M

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from eduge_pickles.rar


Would you like to replace the existing file word_index.pickle
9178153 bytes, modified on 2019-04-13 01:44
with a new one
9178153 bytes, modified on 2019-04-13 01:44

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit q

Program aborted


In [0]:
import pickle

with open('word_index.pickle', 'rb') as handle:
  word_index = pickle.load(handle)
    
with open('reversed_word_index.pickle', 'rb') as handle:
  reversed_word_index = pickle.load(handle)
  
with open('eduge_stopwords_removed.pickle', 'rb') as handle:
  eduge_ds = pickle.load(handle)

In [4]:
# facebook trained word2vec on both commoncrawl and wikipedia. So this model should contain enough representation about our mongolian words.
mongolian_word2vec_download="https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mn.300.bin.gz"
if not exists("cc.mn.300.bin.gz"):
  !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.mn.300.bin.gz
if exists('cc.mn.300.bin.gz'):
  !gunzip cc.mn.300.bin.gz

gzip: cc.mn.300.bin already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [5]:
from gensim.models.wrappers import FastText

word2vec_model = FastText.load_fasttext_format('cc.mn.300.bin')

W0413 23:29:51.869181 140649000748928 ssh.py:33] paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
W0413 23:29:52.127143 140649000748928 word2vec.py:573] Slow version of gensim.models.deprecated.word2vec is being used


In [6]:
print(word2vec_model.most_similar('монгол'))

[('Монгол', 0.6342526078224182), ('монголын', 0.6047513484954834), ('хятад', 0.5558866858482361), ('Монголын', 0.5087883472442627), ('судлалаараа', 0.48851606249809265), ('манай', 0.4853793680667877), ('уйгаржин', 0.4725492596626282), ('угсаатангууд', 0.47093287110328674), ('орос', 0.46463483572006226), ('худам', 0.4609120190143585)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# preparing embedding matrix
import numpy as np

words_not_found = []
embed_dim       = 300
embedding_matrix = np.random.uniform(-1, 1, (len(word_index), embed_dim))
for word, i in word_index.items():
  if i<4:
    continue
  try:
    embedding_vector = word2vec_model[word]
    if (embedding_vector is not None) and len(embedding_vector) > 0:
      embedding_matrix[i] = embedding_vector
  except:
    words_not_found.append(word)
    pass

In [8]:
print(embedding_matrix.shape)
#print(embedding_matrix[5])

(370794, 300)


In [0]:
MAX_LEN = 512

import itertools

for item in eduge_ds:
  item[0] = list(itertools.chain(*item[0]))[:MAX_LEN]

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(eduge_ds, test_size=0.1, random_state=999)

In [0]:
train_data_words  = [i[0] for i in train]
train_label_words = [i[1] for i in train]
test_data_words   = [i[0] for i in test ]
test_label_words  = [i[1] for i in test ]

In [0]:
def encode_news(text):
    return [word_index.get(i, 2) for i in text]
  
train_data = [encode_news(sent) for sent in train_data_words]
test_data  = [encode_news(sent) for sent in test_data_words ]

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=MAX_LEN)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=MAX_LEN)

In [14]:
labels = list(set(test_label_words))
labels

['эрүүл мэнд',
 'эдийн засаг',
 'хууль',
 'улс төр',
 'спорт',
 'боловсрол',
 'байгал орчин',
 'урлаг соёл',
 'технологи']

In [0]:
from sklearn.preprocessing import LabelBinarizer
encoder     = LabelBinarizer()
train_label = transfomed_label = encoder.fit_transform(train_label_words)
test_label  = transfomed_label = encoder.fit_transform(test_label_words )

In [18]:
vocab_size = len(word_index)

sequence_input     = keras.layers.Input(shape=(MAX_LEN,), dtype='int32')
embedded_sequences = keras.layers.Embedding(
    vocab_size, 
    embed_dim , 
    weights=[embedding_matrix], 
    input_length=MAX_LEN, 
    trainable=False)(sequence_input)
x = keras.layers.Conv1D(128, 5, activation='relu')(embedded_sequences)
x = keras.layers.MaxPooling1D(5)(x)
x = keras.layers.Conv1D(128, 5, activation='relu')(x)
x = keras.layers.MaxPooling1D(5)(x)
x = keras.layers.Conv1D(128, 5, activation='relu')(x)
x = keras.layers.MaxPooling1D(15)(x) # global max pooling
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
preds = keras.layers.Dense(len(labels), activation='softmax')(x)

model = keras.models.Model(sequence_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 512, 300)          111238200 
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 508, 128)          192128    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 101, 128)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 97, 128)           82048     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 19, 128)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 15, 128)           82048 

In [0]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
print(len(train_data), len(train_label))
print(len(test_data ), len(test_label) )

partial_index = 3000

x_val = train_data[:partial_index]
partial_x_train = train_data[partial_index:]

y_val = train_label[:partial_index]
partial_y_train = train_label[partial_index:]

68094 68094
7567 7567


In [21]:
epochs = 150
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs  ,
                    batch_size=512 ,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 65094 samples, validate on 3000 samples
Epoch 1/150
65094/65094 [==============================] - 29s 451us/sample - loss: 1.5893 - accuracy: 0.4209 - val_loss: 1.2512 - val_accuracy: 0.5380
Epoch 2/150
65094/65094 [==============================] - 24s 368us/sample - loss: 1.0078 - accuracy: 0.6598 - val_loss: 0.9924 - val_accuracy: 0.6577
Epoch 3/150
65094/65094 [==============================] - 24s 366us/sample - loss: 0.7461 - accuracy: 0.7561 - val_loss: 0.8013 - val_accuracy: 0.7407
Epoch 4/150
65094/65094 [==============================] - 24s 366us/sample - loss: 0.5728 - accuracy: 0.8187 - val_loss: 0.7278 - val_accuracy: 0.7653
Epoch 5/150
65094/65094 [==============================] - 24s 365us/sample - loss: 0.4723 - accuracy: 0.8512 - val_loss: 0.7614 - val_accuracy: 0.7557
Epoch 6/150
65094/65094 [==============================] - 24s 366us/sample - loss: 0.4049 - accuracy: 0.8728 - val_loss: 0.5384 - val_accuracy: 0.8317
Epoch 7/150
65094/65094 [==============

KeyboardInterrupt: ignored

In [22]:
results = model.evaluate(test_data, test_label)
print(results)

7567/7567 [==============================] - 3s 332us/sample - loss: 1.1013 - accuracy: 0.8515
[1.101346154871147, 0.8514603]


In [23]:
data_index   = 12
data_words   = " ".join(test_data_words[data_index])
data_indexes = test_data[data_index]
print(data_words)

predicted = model.predict([[data_indexes]])
print(encoder.classes_[np.argmax(predicted)])

спортын төв ордонд өнөөдөр азийн оюутны аварга шалгаруулах эмэгтэй волейболчдын хоёр дахь удаагийн тэмцээний талаар мэдээлэл хийлээ анхны тэмцээн онд тайландын бангконг хотноо болж хоёрдугаар тэмцээнийг азийн оюутны спортын холбооноос аосх олгосон эрхийн дагуу оны дөрөвдүгээр сарын ны өдрүүдэд монгол улсын нийслэл улаанбаатар хотноо зохион байгуулах тэмцээний эрхийг монгол улс оны тавдугаар сарын хуралдсан аосхны гүйцэтгэх хорооны хурлаар хоёр оронтой өрсөлдөн авчээ уг тэмцээнийг монгол улсад авах талаар мосхолбоо оноос санаачлага гарган хөөцөлдөж эхэлсэн тэмцээний эрхийг авахад муын засгийн газрын санхүүгийн дэмжлэг мэргэжлийн холбоодын ажлын туршлага манай улсын олон улсын нэр хүнд ихээхэн тус хүргэжээ зохион байгуулах хороог с ламбаа удирдаж тэмцээний зохион байгуулах хороог збх эрүүл мэндийн сайдын оны тоот тушаалаар батлаж даргаар уихын гишүүн монголын волейболын холбооны мвх хүндэт ерөнхийлөгч сламбаа ажиллаж збхны орлогч даргаар згхагентлагбтсгын дарга чнаранбаатар збхны нарийн 